# municipal_water_withdrawal_per_capita

Because AQUASTAT doesn't provide this indicator, we want to calculate this indicator using municipal_water_withdrawal and total_population data.

One tricky issue is that in AQUASTAT they don't have data for each year, and data are reported in period. In their FAQ: 

> The latest value that has data of every 5 year period is shown, NOT the average value of that 5 year period. For example, if data are available for the year 2004 and for the year 2006, then the period 2003-2007 value in the AQUASTAT Main Database corresponds to the year 2006. Periods are reported because AQUASTAT data is based on data reported by countries, and data does not exist for every year. Therefore, forcing the data into 5 year periods allows for subsequent calculations (like totals) to be performed.

Judging from their total water use and their total water use per capita, they don't take into account which year in the 5-year period the actual water measurement comes from.

But we want to have data by year, not year range. So we should align the data first.


In [1]:
import pandas as pd
from ddf_utils.model import Datapackage

In [40]:
dp = Datapackage('../../')

In [3]:
municipal_water_withdrawal = dp.get_datapoint_df('municipal_water_withdrawal', ('area', 'year')).compute()

In [4]:
total_population = dp.get_datapoint_df('total_population', ('area', 'year')).compute()

In [5]:
municipal_water_withdrawal.shape

(637, 3)

In [6]:
total_population.shape

(2220, 3)

In [7]:
municipal_water_withdrawal['area'] = municipal_water_withdrawal['area'].astype(str)
total_population['area'] = total_population['area'].astype(str)

In [8]:
municipal_water_withdrawal = municipal_water_withdrawal.set_index(['area', 'year'])
total_population = total_population.set_index(['area', 'year'])

In [9]:
df = municipal_water_withdrawal.join(total_population, how='outer')

In [10]:
df.head(20)

municipal_water_withdrawal  total_population
area        year                                              
afghanistan 1962                         NaN            9344.0
            1967                         NaN           10369.0
            1972                         NaN           11717.0
            1977                         NaN           13056.0
            1982                         NaN           12667.0
            1987                      0.2611           11338.0
            1992                         NaN           13746.0
            1997                         NaN           18034.0
            2000                      0.1542               NaN
            2002                         NaN           21487.0
            2005                      0.2034               NaN
            2007                         NaN           25878.0
            2012                         NaN           29727.0
            2015                         NaN           32527.0
albania     1962                         NaN            1738.0
            1967                         NaN            1999.0
            1972                         NaN            2254.0
            1977                         NaN            2518.0
            1982                         NaN            2788.0
            1987                         NaN            3121.0

In [11]:
df2 = df.copy()

In [12]:
# back fill NaN values for population. In most case this should make 2 series aligned. 
# Because withdrawal data usually not avaliable for the latest year of a period
# TODO: Maybe reindex and create year range manually and compute.

df2['total_population'] = df2['total_population'].fillna(method='bfill')

In [14]:
df2['municipal_water_withdrawal_per_captia'] = df2['municipal_water_withdrawal'] / df2['total_population'] * (10**6)

In [16]:
df2['municipal_water_withdrawal_per_captia'].dropna()

area                              year
afghanistan                       1987     23.028753
                                  2000      7.176432
                                  2005      7.859958
albania                           1990     30.854674
                                  2000    176.240794
                                  2005    186.416473
algeria                           1980     32.076205
                                  1990     41.205254
                                  2000     53.798062
                                  2012     80.664548
angola                            1987      6.513708
                                  2000     15.270019
                                  2005     16.654504
antigua_and_barbuda               1990     47.296232
                                  2005     62.796209
                                  2012     80.835298
argentina                         1995    127.253446
                                  2011    138.971374
armenia

In [18]:
(df2['municipal_water_withdrawal_per_captia'].dropna()
.to_csv('../../ddf--datapoints--municipal_water_withdrawal_per_captia--by--area--year.csv'))

In [41]:
concepts = dp.concepts

In [42]:
cdf = concepts[concepts.concept_type == 'measure'].copy()

In [43]:
cdf[cdf.concept == 'area_equipped_for_irrigation_by_direct_use_of_treated_municipal_wastewater']

,concept,concept_type,name,variable_id
26,area_equipped_for_irrigation_by_direct_use_of_...,measure,Area equipped for irrigation by direct use of ...,4533.0


In [44]:
cdf = cdf.set_index('concept')

In [45]:
cdf['variable_id'] = cdf['variable_id'].map(lambda x: str(int(float(x))))

In [46]:
cdf.loc['municipal_water_withdrawal_per_captia'] = ['measure', 'Municipal Water Withdrawal Per Captia', '']

In [47]:
cdf.sort_index().to_csv('../../ddf--concepts--continuous.csv')